# Image Meta Workflow
This notebook outlines some workflow details how to use the Image Metadata maintenance that can be done with the `image_meta` package. For an Introduction check out the [IMAGE_META Notebook](./IMAGE_META.ipynb)
It requires a separate installation of **[EXIFTOOL](https://exiftool.org)**

In [ ]:
import os
import imp
from image_meta import persistence
from image_meta import controller
from image_meta import exif
from image_meta import util
from image_meta import geo
imp.reload(persistence)
imp.reload(controller)
imp.reload(exif)
imp.reload(util)
imp.reload(geo)
from image_meta.persistence import Persistence
from image_meta.controller import Controller
from image_meta.exif import ExifTool
from image_meta.util import Util
from image_meta.geo import Geo

# Workflow explained

**Let's get started by copying the sample files:**

In [ ]:
# help on rename/copy function (check for details)
help(Persistence.copy_rename)

# Define variables


In [ ]:
# define filepath 
initial_fp = os.getcwd()
print(f"Current Filepath: {initial_fp}")

# load the work settings json
# settings_json = os.path.join(initial_fp,"work_settings.json")

# if not os.path.isfile(settings_json):
#     print(f"{settings_json} is not a valid file")
#     work_dir = None
# else:
#     settings = Persistence.read_json(settings_json)
#     print(f"\n--- Settings in file {settings_json} ---")
#     Util.print_dict_info(settings)
# print(f"Working Directory is valid directory: {os.path.isdir(work_dir)}")

# for the sample work dir is direct subpath
work_dir = os.path.join(initial_fp,"Sample")
print(f"Work Dir: {work_dir}")

In [ ]:
# files customized to your own requirements go into 00_OWN_STUFF
work_dir_ownstuff = os.path.join(work_dir,"00_OWN_STUFF")
# files with GPS data
work_dir_gps = os.path.join(work_dir,"10_RAW_GPS")
# files without GPS data
work_dir_nogps = os.path.join(work_dir,"10_RAW_NOGPS")
# files with exif metadata
work_dir_meta = os.path.join(work_dir,"10_RAW_WITH_META")
# original metadata
meta_dir = os.path.join(work_dir,"00_META")
# work directory
target_dir = os.path.join(work_dir,"img_test")
img_dir = target_dir
print(f"Original Jupyter path: {initial_fp}, \nTarget directory {work_dir}")

# copy all test file to target work dir
# check the docuemtation
regex_filter=None
regex_subst=None
s_subst=""
debug=True
save=True
# copy images amnd meta files
dirs = [work_dir_ownstuff,work_dir_gps,work_dir_nogps,meta_dir,work_dir_meta]
for d in dirs:
    Persistence.copy_rename(d,target_dir,
                            regex_filter=regex_filter,regex_subst=regex_subst,
                            s_subst=s_subst,debug=debug,save=save)  


**Create a Metdata Template File**

In [ ]:
# will be created in work directory / you can delete all entries with the "INFO_" prefix
control_fp = Controller.create_param_template(filepath=img_dir,name="metadata.tpl",showinfo=debug)
print(f"Creation of Template file: {control_fp}")

**That's how the template file looks like ... it also contains descriptions (INFO_...) that you can delete afterwards**

In [ ]:
# the control file is json
control_template = Persistence.read_json(control_fp)
# here we print out the json key / value pairs
# Items starting with "INFO_" are meant for information only
Util.print_dict_info(control_template)

By means of this file you'll bundle data required to do the metadata handling. The `tpl` File references as auxiliary files as well control parameters in order to execute the metadata handling. 

You can copy the sample file `..\image_meta_sample\Sample\img_test\metadata_sample.tpl` and adjust the path in WORKING_DIRECTORY pointing to your `..image_meta_sample\Sample\img_test` directory. (mind the slash `/` to be used for directory for windows)

**CAVEAT** Check that Files are UTF8 Files

In [ ]:
# here's the ref to your own control parameter file / adjust as needed
showinfo = True
control_fp = os.path.join(img_dir,"metadata_own.tpl")
print("--- CONTROL PARAMETERS ---")
control_params = Controller.read_params_from_file(filepath=control_fp,showinfo=showinfo)

And there's also a file that provides default values for EXIF keywords (Keys correspond to Exif Keys)

In [ ]:
fp_work_dir = control_params["WORK_DIR"]
print(f"-----\nWorking Directory {fp_work_dir}\n-----")
fp_exif_metadata = control_params["META_FILE"]
print(f"Exif Metafile {fp_exif_metadata}\n-----")

# read its contents
try:
    meta_raw = Persistence.read_file(fp_exif_metadata)
    meta_exif = ExifTool.arg2dict(meta_raw)
except:
    meta_exif = {}

# print contents    
Util.print_dict_info(meta_exif)

Reading the metadata hierarchy file (needs to be UTF8)

In [ ]:
fp_keyword_hier = control_params["KEYWORD_HIER_FILE"]
# raw data from file
hier_raw = Persistence.read_file(fp_keyword_hier)
print(f"---\nKeyword Hierarchy File\n{fp_keyword_hier}\n---")
# transforming raw text data into hierarcy dictionary
keyword_hier = ExifTool.create_metahierarchy_from_str(hier_raw)
Util.print_dict_info(keyword_hier)

And we also get the GPX geodata ...

In [ ]:
fp_gpx = control_params["GPX_FILE"]
print(f"-----\nGPX File {fp_gpx}\n-----")
gpx = Persistence.read_gpx(gpsx_path=fp_gpx,debug=True)

With the gpx file, we can get the geo coordinates for a given time stamp

In [ ]:
# keys are utc timestamps
gpx_ts_list = sorted(gpx.keys())
# sample datetime and timestamp
sample_time = "2020-06-15 11:50:45+02:00"
dt = Util.get_localized_datetime(dt_in=sample_time,as_timestamp=False)
ts = Util.get_localized_datetime(dt_in=sample_time,as_timestamp=True)
# get nearest timestamp on gps track
ts_gps = gpx_ts_list[Util.get_nearby_index(ts,sorted_list=gpx_ts_list)]
print(f"Datetime {dt} Timestamp {ts} Timestamp GPS Track {ts_gps}")
# with the timestamp we can access the GPS Data
gpx_dict = gpx[ts_gps]
Util.print_dict_info(gpx_dict)
# show gps map
print("Click to show location on OSM")
print(Geo.latlon2osm((gpx_dict["lat"],gpx_dict["lon"])))

Default GPS LatLon Data

In [ ]:
default_latlon = control_params["DEFAULT_LATLON"]
print(f"-----\nDefault LatLon coordinates {default_latlon}\n-----")

From LatLon, Geo Reverse Data can be retrieved from web

In [ ]:
geo_dict = Geo.geo_reverse_from_nominatim(default_latlon,debug=False)

Show the data

In [ ]:
Util.print_dict_info(geo_dict)

**Follow Up actions**

The processing of metadata by means of Exiftool is sketched in the other Jupyter Notebook. 

This is just meant for information. Putting all data in a Control data file and let do the controller class do the orchestration of all data processing will execute the whole metadata processing.

The workflow is as follows:
* Load template data, gps data, keyword hierarchy
* Load image files
* For each image, read technical metadata (to be used as additional EXIF data)
* For each image, check if gps data can be assigned (depending on creation date). If not, default GPS data are assigned
* If GPS data can be found, execute a georeverse search and save the data to a separate file
* For each image, assign template metadata, keyword hierarchy, gps data, and geometadata from georeverse search
* For each image, save a file with all augmented data to a separate metadata file
* For each image, change metadata from the metadata file generated in previous step

# Workflow Steps with Controller Module

Having shown some of the files required to process images, we can put together all items

Control data file needs to be defined / referenced as filepath (execute cells above)

In [ ]:
# here's the ref to your own control parameter file / adjust as needed
showinfo = False
control_fp = os.path.join(img_dir,"metadata_own.tpl")

In [ ]:
print(f"Filepath to Control File: {control_fp}")

Here we read the file again ... parameters will be used to control execution / find files for processing

In [ ]:
print("--- CONTROL PARAMETERS ---")
control_params = Controller.read_params_from_file(filepath=control_fp,showinfo=showinfo)
control_params

`Controller.prepare_execution` checks for all files referenced in `ontrol_params` and blends together data.

In [ ]:
showinfo = True
augmented_params = Controller.prepare_execution(template_dict=control_params,showinfo=showinfo)   

Compiled parameters wil be used to create gelocation (.geo) / control files with metadata (.meta) that will be written as EXIF data

In [ ]:
verbose = False # detailed output
Controller.prepare_img_write(params=augmented_params,debug=showinfo,verbose=verbose)

In [ ]:
# Todo Validate Method - Checking for plausible